In [1]:
import pandas as pd 

agrega = pd.read_csv('gs://trabalho-final-pdm/silver/agregados_2016.csv', storage_options={"token": "google_default"})
# Preparação: adicionar uma coluna de renda média (V005 + V009)
agrega['RENDA'] = agrega['V005'] + agrega['V009']

In [2]:
agrega.head(2)

,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,Nome_do_subdistrito,Situacao_setor_x,V005,V009,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,RENDA
0,2016,GOIANIA,133,146,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,1,COLEGIO ESTADUAL MAJOR OSCAR ALVEOLOS,U.T.P. PARQUE AMAZONIA,1.0,1497.56,971.51,0.540136,0.521088,0.236735,2469.07
1,2016,GOIANIA,2,42,2,1,VEREADOR,MARIO MIGUEL GHANNAM,19234,3,ESC TEC DE COMERCIO DE CAMPINAS E COLEGIO PRIN...,U.T.P. COIMBRA,1.0,1497.84,1264.87,0.165548,0.536913,0.201342,2762.71


In [3]:
# 1. NR_VOTAVEL com maior número de votos por Nome_do_subdistrito
# 2. NM_VOTAVEL com maior número de votos por Nome_do_subdistrito
nr_votavel_subdistrito = agrega.groupby(['Nome_do_subdistrito', 'NR_VOTAVEL', 'NM_VOTAVEL', 'Proporcao_negros_pardos_indigenas', 'Proporcao_feminino', 'Proporcao_18_39'])['QT_VOTOS'].sum().reset_index()
nr_votavel_max = nr_votavel_subdistrito.loc[nr_votavel_subdistrito.groupby('Nome_do_subdistrito')['QT_VOTOS'].idxmax()]

# Partidos políticos mais bem votados por bairro
# Candidato com maior numero de votos por bairro
nr_votavel_max.sort_values('QT_VOTOS')

,Nome_do_subdistrito,NR_VOTAVEL,NM_VOTAVEL,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,QT_VOTOS
8001,U.T.P. JARDIM AMERICA,96,VOTO NULO,0.426829,0.493902,0.225610,10
15367,U.T.P. PARQUE BOM JESUS,90900,SEBASTIAO DE SOUSA PORTO FILHO,0.709369,0.534417,0.216061,35
22342,U.T.P. VILA RIZZO,96,VOTO NULO,0.591331,0.513932,0.286378,47
13362,U.T.P. NORTE FERROVIARIO,96,VOTO NULO,0.363225,0.557971,0.214674,72
22532,VILA RICA,55555,PRISCILLA NORGANN DE SOUSA ROCHA,0.626374,0.520879,0.219780,86
478,U.T.P. ALTO DA GLORIA E REDENCAO,96,VOTO NULO,0.257403,0.544419,0.280182,100
2392,U.T.P. CAMPINAS,44444,JORGE KAJURU REIS DA COSTA NASSER,0.434524,0.517857,0.273810,149
5719,U.T.P. CHACARAS SAO JOAQUIM,27015,IZABEL MARTINE FLORA LUSO,0.681941,0.536388,0.231806,154
17262,U.T.P. RIVIERA/AGUA BRANCA,96,VOTO NULO,0.473190,0.564343,0.199732,179
4688,U.T.P. CELINA PARK/RECREIO DOS FUNCIONARIOS,96,VOTO NULO,0.470650,0.519916,0.227463,187


In [4]:
from google.cloud import bigquery
import pandas as pd

# Configuração
client = bigquery.Client()
table_ref = "pdm-class-2024.gold.infos-por-bairro-2016"

# Carregar o DataFrame para o BigQuery
job = client.load_table_from_dataframe(nr_votavel_max, table_ref)
job.result()

print(f"Tabela {table_ref} exportada com sucesso!")


Tabela pdm-class-2024.gold.infos-por-bairro-2016 exportada com sucesso!


In [5]:
# 3. NM_VOTAVEL com maior número de votos por renda
# 4. NR_VOTAVEL com maior número de votos por renda
# Ordenar por renda para pegar as rendas mais altas e baixas
nm_votavel_renda = agrega.groupby(['RENDA','NR_VOTAVEL','NM_VOTAVEL', 'Proporcao_negros_pardos_indigenas', 'Proporcao_feminino', 'Proporcao_18_39'])['QT_VOTOS'].sum().reset_index()
renda_sorted = nm_votavel_renda.sort_values(by='RENDA')

# Candidato e partido com maior quantidade de votos por renda (renda baixa e alta)
renda_sorted

,RENDA,NR_VOTAVEL,NM_VOTAVEL,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,QT_VOTOS
0,1070.39,12,PARTIDO DEMOCRATICO TRABALHISTA,0.626374,0.520879,0.219780,1
32,1070.39,43249,THIAGO AZEVEDO RAMOS DA SILVA,0.626374,0.520879,0.219780,1
31,1070.39,43000,EDUARDO PEREIRA DE SOUZA,0.626374,0.520879,0.219780,2
30,1070.39,36153,ROMARIO BARBOSA POLICARPO,0.626374,0.520879,0.219780,9
14,1070.39,18888,SUMAIA RODRIGUES DE SOUZA,0.626374,0.520879,0.219780,3
...,...,...,...,...,...,...,...
22529,17065.38,90700,PAULO CESAR DA SILVA,0.158405,0.513390,0.156695,168
22528,17065.38,90633,ANTONIO CARLOS DE JESUS RODRIGUES,0.158405,0.513390,0.156695,1
22478,17065.38,65123,ELENIRA TATIANA LEMOS VIEIRA CHADUD,0.158405,0.513390,0.156695,174
22451,17065.38,55122,JOSE RIBAMAR FERNANDES DE OLIVEIRA,0.158405,0.513390,0.156695,1


In [6]:
from google.cloud import bigquery
import pandas as pd

# Configuração
client = bigquery.Client()
table_ref = "pdm-class-2024.gold.infos-por-renda-2016"

# Carregar o DataFrame para o BigQuery
job = client.load_table_from_dataframe(renda_sorted, table_ref)
job.result()

print(f"Tabela {table_ref} exportada com sucesso!")


Tabela pdm-class-2024.gold.infos-por-renda-2016 exportada com sucesso!
